In [1]:
import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB as BOHB
import pickle

import numpy as np



In [2]:
import sys
sys.path.insert(0, '..')


import System as SEQ
%run block_EQS.ipynb


---------------------- SCIANN 0.6.4.5 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.3.0 
Python Version: 3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)] 



In [3]:
import Pinn_Worker

In [4]:
test_gridObj = SEQ.Grid(4,{'x':0,'y':1,'z':2,'t':3},[[[-2,2],[-2,2],[-2,2],[0,5]]],5)

configspecs = {
    'denspt':[5,12],
    'numNeurons':[10,100],
    'numLayers':[2,50],
    'activator': ['elu','relu','selu','swish',
                 'tanh','Addons>gelu','Addons>mish',
                          'Addons>softshrink'],
    'loss':['mae','mse'],
    'optimizer':['Adam','RMSprop','SGD','Nadam','Ftrl'],
    'batch_size':[5000,10000,15000,25000],
    'initial_lr':[1e-4,1e-2]
}


In [6]:
# Step 1: Start a nameserver
# Every run needs a nameserver. It could be a 'static' server with a
# permanent address, but here it will be started for the local machine with the default port.
# The nameserver manages the concurrent running workers across all possible threads or clusternodes.
# Note the run_id argument. This uniquely identifies a run of any HpBandSter optimizer.

numWorkers = 1
experiment_name = 'block_exp'
run_id = 'block_GridSearch_1'

NS = hpns.NameServer(run_id=run_id, host='127.0.0.1', port=None)
NS.start()

# Step 2: Start a worker
# Now we can instantiate a worker, providing the mandatory information
# Besides the sleep_interval, we need to define the nameserver information and
# the same run_id as above. After that, we can start the worker in the background,
# where it will wait for incoming configurations to evaluate.


workers=[]
for i in range(1,numWorkers+1):
    print('Creating worker')
    w = Pinn_Worker.PINN_Worker(
    valData = valData,
    test_gridObj=test_gridObj,
    PDESystem=mySys,
    configspecs = configspecs,
    valFromFEM=False,
    experiment_name = experiment_name,
    nameserver='127.0.0.1',
    run_id=run_id,
    id=i)
    
    print('Worker created, starting it')
    w.run(background=True)
    print('Worker running, adding it to the list')
    workers.append(w)
    print('Worker added')
    


Creating worker
Creating client
Existing experiment. Recovering id.
Applying validation data to test grid
Saving PDE and config
Worker ready
Worker created, starting it
Worker running, adding it to the list
Worker added


14:46:10 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x1d49b007f88; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
14:46:10 WORKER: No dispatcher found. Waiting for one to initiate contact.
14:46:10 WORKER: start listening for jobs


In [7]:
# bohb = BOHB(  configspace = w.get_configspace(),
#               run_id = 'example1', nameserver='127.0.0.1',
#               min_budget=10, max_budget=1000
#            )
# res = bohb.run(n_iterations=25
#               )

bohb = BOHB(  configspace = w.get_configspace(),
              run_id = run_id,
              min_budget=10, max_budget=500
           )
res = bohb.run(n_iterations=5, min_n_workers=numWorkers)


14:46:10 wait_for_workers trying to get the condition
14:46:10 DISPATCHER: started the 'discover_worker' thread
14:46:10 DISPATCHER: started the 'job_runner' thread
14:46:10 DISPATCHER: Pyro daemon running on localhost:52261
14:46:10 DISPATCHER: Starting worker discovery
14:46:10 DISPATCHER: Found 1 potential workers, 0 currently in the pool.
14:46:10 DISPATCHER: discovered new worker, hpbandster.run_block_GridSearch_1.worker.DESKTOP-NKTB8UL.7940.13296
14:46:10 HBMASTER: number of workers changed to 1
14:46:10 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
14:46:10 HBMASTER: only 1 worker(s) available, waiting for at least 1.
14:46:10 adjust_queue_size: lock accquired
14:46:10 HBMASTER: adjusted queue size to (0, 1)
14:46:10 DISPATCHER: Finished worker discovery
14:46:10 DISPATCHER: A new worker triggered discover_worker
14:46:10 DISPATCHER: Trying to submit another job.
14:46:10 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
14:46:10 Eno


Total samples: 81920 
Batch size: 15000 
Total batches: 6 



14:47:10 DISPATCHER: Starting worker discovery
14:47:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
14:47:10 DISPATCHER: Finished worker discovery
14:47:23 WORKER: done with job (0, 0, 0), trying to register it.
14:47:23 WORKER: registered result for job (0, 0, 0) with dispatcher
14:47:23 DISPATCHER: job (0, 0, 0) finished
14:47:23 DISPATCHER: register_result: lock acquired
14:47:23 DISPATCHER: job (0, 0, 0) on hpbandster.run_block_GridSearch_1.worker.DESKTOP-NKTB8UL.7940.13296 finished
14:47:23 job_id: (0, 0, 0)
kwargs: {'config': {'activator': 'sigmoid', 'batch_size': 15000, 'denspt': 4, 'initial_lr': 0.003804834845191026, 'loss': 'mse', 'numLayers': 46, 'numNeurons': 61, 'optimizer': 'Adam'}, 'budget': 18.51851851851852, 'working_directory': '.'}
result: None
exception: Traceback (most recent call last):
  File "C:\Users\Felix\anaconda3\envs\tf_gpu\lib\site-packages\hpbandster\core\worker.py", line 206, in start_computation
    result = {'result': self.compute(*a


Total samples: 414720 
Batch size: 10000 
Total batches: 42 



14:48:10 DISPATCHER: Starting worker discovery
14:48:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
14:48:10 DISPATCHER: Finished worker discovery
14:49:10 DISPATCHER: Starting worker discovery
14:49:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
14:49:10 DISPATCHER: Finished worker discovery
14:50:10 DISPATCHER: Starting worker discovery
14:50:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
14:50:10 DISPATCHER: Finished worker discovery
14:50:17 WORKER: done with job (0, 0, 1), trying to register it.
14:50:17 WORKER: registered result for job (0, 0, 1) with dispatcher
14:50:17 DISPATCHER: job (0, 0, 1) finished
14:50:17 DISPATCHER: register_result: lock acquired
14:50:17 DISPATCHER: job (0, 0, 1) on hpbandster.run_block_GridSearch_1.worker.DESKTOP-NKTB8UL.7940.13296 finished
14:50:17 job_id: (0, 0, 1)
kwargs: {'config': {'activator': 'sigmoid', 'batch_size': 10000, 'denspt': 6, 'initial_lr': 0.0096399433434633, 'loss': 'mae', 

...


Total samples: 200000 
Batch size: 15000 
Total batches: 14 



15:06:10 DISPATCHER: Starting worker discovery
15:06:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
15:06:10 DISPATCHER: Finished worker discovery
15:06:35 WORKER: done with job (0, 0, 13), trying to register it.
15:06:35 WORKER: registered result for job (0, 0, 13) with dispatcher
15:06:35 DISPATCHER: job (0, 0, 13) finished
15:06:35 DISPATCHER: register_result: lock acquired
15:06:35 DISPATCHER: job (0, 0, 13) on hpbandster.run_block_GridSearch_1.worker.DESKTOP-NKTB8UL.7940.13296 finished
15:06:35 job_id: (0, 0, 13)
kwargs: {'config': {'activator': 'relu', 'batch_size': 15000, 'denspt': 5, 'initial_lr': 0.006436430583913265, 'loss': 'mae', 'numLayers': 43, 'numNeurons': 20, 'optimizer': 'SGD'}, 'budget': 18.51851851851852, 'working_directory': '.'}
result: {'loss': 353.7963075470491, 'info': {'L1': 353.7963075470491, 'L2': 616.5765643304507, 'MAX': 2.7583236694335938, 'TrainTime': 58.546875}}
exception: None

15:06:35 job_callback for (0, 0, 13) started
15:06:35 D


Total samples: 414720 
Batch size: 10000 
Total batches: 42 



15:07:10 DISPATCHER: Starting worker discovery
15:07:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
15:07:10 DISPATCHER: Finished worker discovery
15:08:10 DISPATCHER: Starting worker discovery
15:08:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
15:08:10 DISPATCHER: Finished worker discovery
15:09:10 DISPATCHER: Starting worker discovery
15:09:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
15:09:10 DISPATCHER: Finished worker discovery
15:10:10 DISPATCHER: Starting worker discovery
15:10:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
15:10:10 DISPATCHER: Finished worker discovery
15:11:10 DISPATCHER: Starting worker discovery
15:11:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
15:11:10 DISPATCHER: Finished worker discovery
15:12:10 DISPATCHER: Starting worker discovery
15:12:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
15:12:10 DISPATCHER: Finished worker discover

KeyboardInterrupt: 

In [ ]:
np.save('HpBandster_Results',res,allow_pickle=True)
bohb.shutdown(shutdown_workers=True)
NS.shutdown()

15:16:10 DISPATCHER: Starting worker discovery
15:16:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
15:16:10 DISPATCHER: Finished worker discovery
15:17:10 DISPATCHER: Starting worker discovery
15:17:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
15:17:10 DISPATCHER: Finished worker discovery
15:18:10 DISPATCHER: Starting worker discovery
15:18:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
15:18:10 DISPATCHER: Finished worker discovery
15:19:10 DISPATCHER: Starting worker discovery
15:19:10 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
15:19:10 DISPATCHER: Finished worker discovery
